In [8]:
import ocrmypdf
from pdf2docx import parse, Converter
import PyPDF2
from PyPDF2 import PdfFileWriter, PdfFileReader
import sys
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders


receiver_name="*****"     #user name and email id
receiver_id="*******"

#function to convert to OCR pdf
def non_ocr_to_ocr(file_path, save_path):
    ocrmypdf.ocr(file_path, save_path, skip_text=True)
    print("converted to ocr pdf")
    pdf_encrypt(save_path)
   
#function to encrypt Pdf
def pdf_encrypt(pdf_file):
    out = PdfFileWriter()  #create a pdfFileWriter object
    file = PdfFileReader(pdf_file)  #Open PDF file with the PdfFileReader
    num = file.numPages  #Get number of pages in original fil
    
    #Iterate through every page of the original file and add it to our new file.
    for idx in range(num):
        page = file.getPage(idx)  #Get the page at index idx
        out.addPage(page)        #Add it to the output file
    
    #function to generate password for encrypted file
    def pwd_generate(name):
        l=name.split(" ")
        pwd=""
        if len(l)>1:
            pwd+=l[0][0:2]
            pwd+=l[1][0:2]
        else:
            pwd+=l[0][0:4]
        return pwd
    
    #password generated and stored in password variable
    password = pwd_generate(receiver_name)
    out.encrypt(password)   #Encrypt the new file with the entered password
    
    #Open a new file "ocr_pdf_encrypted.pdf"
    with open("ocr_pdf_encrypted.pdf", "wb") as f:
        out.write(f)   #Write encrypted PDF to this file
    print("Pdf file enctypted")
    SendMail(receiver_name,receiver_id,password,"ocr_pdf_encrypted.pdf")


#function to send email to the user along with the pdf attachment
def SendMail(name,id,password,pdfname):
    sender = 'reenadummy@gmail.com'   #sender email and password
    sender_password = 'Reena@123'
    body = "Hello "+name+"\n\nThe PDF file is converted to OCR PDF file and encrypted successfully.\nPassword is shared below\n\npassword: "+password+"\n\n\nRegards"
    
    #setup MIME
    message = MIMEMultipart()
    message['From'] = sender
    message['To'] = id
    message['Subject'] = 'PDF file process'
    message.attach(MIMEText(body, 'plain'))
    
    #open file in binary
    binary_pdf = open(pdfname, 'rb')
    payload = MIMEBase('application', 'octate-stream', Name=pdfname)
    payload.set_payload((binary_pdf).read())
    
    #encodeing the binary into base64
    encoders.encode_base64(payload)
    
    #add header with pdf name
    payload.add_header('Content-Decomposition', 'attachment', filename=pdfname)
    message.attach(payload)

    session = smtplib.SMTP('smtp.gmail.com', 587)  #use gmail port
    session.starttls()   #enable security
    session.login(sender, sender_password) #gogging in
    text = message.as_string()
    session.sendmail(sender, id, text)
    session.quit()
    print('Mail Sent')
    
non_ocr_to_ocr("non_ocr_pdf.pdf","ocr_pdf.pdf")



Scanning contents: 100%|███████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  3.06page/s]
[INFO] Start processing 4 pages concurrently
OCR:  10%|███████▎                                                                 | 0.5/5.0 [00:01<00:16,  3.59s/page][INFO] skipping all processing on this page
[INFO] skipping all processing on this page
[INFO] skipping all processing on this page
OCR:  70%|███████████████████████████████████████████████████                      | 3.5/5.0 [00:01<00:03,  2.53s/page][INFO] skipping all processing on this page
[INFO] skipping all processing on this page
OCR: 100%|█████████████████████████████████████████████████████████████████████████| 5.0/5.0 [00:02<00:00,  2.22page/s]
[INFO] Postprocessing...
PDF/A conversion: 100%|████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 19.09page/s]
JPEGs: 0image [00:00, ?image/s]
JBIG2: 0item [00:00, ?item/s]
[INFO] Optimize ratio: 1.00 savings: 0.0%
[INFO] O

converted to ocr pdf
Pdf file enctypted
Mail Sent
